# Setting

In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import re
import requests as rq
from bs4 import BeautifulSoup as bs

In [2]:
# 크롬 드라이버 설치
driver_path = ChromeDriverManager().install()
print('크롬 드라이버 설치 경로 : ', driver_path)

크롬 드라이버 설치 경로 :  C:\Users\njh26\.wdm\drivers\chromedriver\win64\122.0.6261.128\chromedriver-win32/chromedriver.exe


In [3]:
# 수집하고자 하는 상품의 URL
# url = 'https://www.coupang.com/vp/products/21277951?itemId=83185355&vendorItemId=74185081497&src=1032001&spec=10305199&addtag=400&ctag=21277951&lptag=I83185355&itime=20240315115333&pageType=PRODUCT&pageValue=21277951&wPcid=17104712136363137870172&wRef=cr.shopping.naver.com&wTime=20240315115333&redirect=landing&mcid=96a64a71f4a84950a8e05f2294b21678&isAddedCart='
# url = 'https://www.coupang.com/vp/products/7608804797?itemId=20146298818&vendorItemId=87239403818&pickType=COU_PICK&q=%EC%97%90%EC%96%B4%ED%8C%9F+%ED%94%84%EB%A1%9C&itemsCount=36&searchId=74335e87972a42f7b6c14e7e6f3c6cf6&rank=0&isAddedCart='
url = 'https://www.coupang.com/vp/products/111269023?itemId=335162862&vendorItemId=3822330082&pickType=COU_PICK&q=%EB%96%A1%EB%B3%B6%EC%9D%B4&itemsCount=36&searchId=c3c994381a9d432689ce7e991eaeddfc&rank=1&isAddedCart='

# 브라우저 인스턴스 생성, url 오픈
driver = webdriver.Chrome(service=Service(driver_path))
driver.get(url)
driver.implicitly_wait(10)

# 상품명 가져오기
product_name = driver.find_element(By.CLASS_NAME, 'prod-buy-header__title').text
### df.to_csv에서 오류가 발생한 경우 위 코드에서 문자를 추가로 변환해야 함
print('상품명 : ', product_name)

# 드라이버 종료
driver.quit()

상품명 :  풀무원 밀 누들 국물떡볶이 2인분, 423.5g, 1개


# Crawling

In [4]:
# 수집할 리뷰 개수
n = 100

# 페이지 수
pages = n*10  # n개까지 리뷰를 충분히 수집할 수 있도록 페이지 수 설정

# URL 주소 재가공
'''https://github.com/JaehyoJJAng/Coupang-Review-Crawling에서 인용하여 수정하였습니다.'''
prod_code = url.split('products/')[-1].split('?')[0]
urls = [
    f'https://www.coupang.com/vp/product/reviews?productId={prod_code}&page={page}&size=5&sortBy=DATE_DESC&ratings=&q=&viRoleCode=3&ratingSummary=true'
    for page in range(1, pages + 1)]

# 접속 차단 우회
headers = {
    "authority": "www.coupang.com",
    "method": "GET",
    "accept": "*/*",
    "accept-encoding": "gzip, deflate, br",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.104 Whale/3.13.131.36 Safari/537.36",
    "sec-ch-ua-platform": "macOS",
    "sec-ch-ua-mobile": "?0",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "cookie": "PCID=31489593180081104183684; _fbp=fb.1.1644931520418.1544640325; gd1=Y; X-CP-PT-locale=ko_KR; MARKETID=31489593180081104183684; sid=03ae1c0ed61946c19e760cf1a3d9317d808aca8b; overrideAbTestGroup=%5B%5D; x-coupang-origin-region=KOREA; x-coupang-accept-language=ko_KR;"
    }

In [5]:
# 리뷰를 저장할 리스트 생성
reviews = []
ratings = []
days = []

# 크롤링
while len(reviews) < n:
    for url in urls:
            session = rq.Session()
            headers['referer'] = url
            response = session.get(url=url, headers=headers)
            html = response.text
            soup = bs(html, 'html.parser')

            # 리뷰 박스
            article_lenth = len(soup.select('article.sdp-review__article__list'))

            # 1~5번 리뷰 수집
            for idx in range(article_lenth):
                articles = soup.select('article.sdp-review__article__list')

                # 리뷰 본문
                review_content = articles[idx].select_one('div.sdp-review__article__list__review > div')
                if review_content == None:
                    continue   # 리뷰 본문이 없는 경우 다음 리뷰로 넘어감
                else:
                    reviews.append(re.sub('[\n\t]', '', review_content.text.strip()))

                    # 리뷰 본문이 있는 경우에만 별점, 리뷰작성일 수집
                    # 별점
                    rating = articles[idx].select_one('div.sdp-review__article__list__info__product-info__star-orange')
                    if rating == None:
                        ratings.append(0)
                    else:
                        ratings.append(int(rating.attrs['data-rating']))

                    # 리뷰작성일
                    day = articles[idx].select_one('div.sdp-review__article__list__info__product-info__reg-date')
                    days.append(day.text)
                    
            if len(reviews) >= n:  # 리뷰 개수가 n 이상이면 루프 종료
                break

In [6]:
# 크롤링 결과
df = pd.DataFrame({'reviews': reviews, 'ratings': ratings, 'days': days})
print(f'\n수집한 데이터는 다음과 같습니다. \n {df.head(4)}')

df.to_csv(f"{product_name}_{n}.csv", encoding='utf-8-sig')


수집한 데이터는 다음과 같습니다. 
                                              reviews  ratings        days
0                                           편해서 좋아요~        4  2024.03.15
1                            가격 변동이 심함그래도 계속 사 먹음맛있음        5  2024.03.14
2  초등아이들이 떡볶이 러버라서 쿠팡에서 가장 인기제품으로. 구매해보았습니다!! -배송...        3  2024.03.14
3      살짝 매콤한데 맛있어요초등아이도 잘먹어요간식용으로 딱 좋아서여러번 재주문 했습니다        5  2024.03.13
